# Tutorial 3
In this tutorial, we are going to directly train a simple SNN with a single hidden layer using EventProp on the MNIST dataset, converted to a latency spike code.

Clearly, this is far from a state of the art architecture, but it still achieves 96% accuracy on MNIST.

## Install
Download wheel file

In [1]:
if "google.colab" in str(get_ipython()):
    !gdown 1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
    !pip install pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
    %env CUDA_PATH=/usr/local/cuda

    !rm -rf /content/ml_genn-ml_genn_2_2_1
    !wget https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_2_1.zip
    !unzip -q ml_genn_2_2_1.zip
    !pip install ./ml_genn-ml_genn_2_2_1/ml_genn

Downloading...
From: https://drive.google.com/uc?id=1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
To: /content/pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
100% 8.38M/8.38M [00:00<00:00, 113MB/s]
Processing ./pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
pygenn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
env: CUDA_PATH=/usr/local/cuda
--2024-10-14 17:15:19--  https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_2_1.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_2_1 [following]
--2024-10-14 17:15:19--  https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_2_1
Resolving codeload.github.com (codeload.github.com)... 140.82.116.10
Connecting to codeload.github.com (codeload.github.com)|140.82.116.

## Install MNIST package

In [12]:
!pip install mnist

## Build model
Import standard modules and required mlGeNN classes

In [13]:
import mnist
import numpy as np
import matplotlib.pyplot as plt

from ml_genn import InputLayer, Layer, SequentialNetwork
from ml_genn.callbacks import Checkpoint
from ml_genn.compilers import EventPropCompiler, InferenceCompiler
from ml_genn.connectivity import Dense
from ml_genn.initializers import Normal
from ml_genn.neurons import LeakyIntegrate, LeakyIntegrateFire, SpikeInput
from ml_genn.optimisers import Adam
from ml_genn.serialisers import Numpy
from ml_genn.synapses import Exponential

from ml_genn.utils.data import (calc_latest_spike_time, linear_latency_encode_data)

from ml_genn.compilers.event_prop_compiler import default_params


##Parameters

Define some model parameters


In [14]:
NUM_INPUT = 28 * 28
NUM_HIDDEN = 128
NUM_OUTPUT = 10
BATCH_SIZE = 128

## Latency encoding
There are numerous ways to encode images using spikes but here we are going to emit a single spike for each neuron at a time proportional the each pixel's grayscale.

In [5]:
mnist.datasets_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"
train_spikes = linear_latency_encode_data(mnist.train_images(), 20.0)

## Network definition
Because our network is entirely feedforward, we can define it as a ``SequentialNetwork`` where each layer is automatically connected to the previous layer. As we have converted the MNIST dataset to spikes, we will use a ``SpikeInput`` to inject these directly into the network. For our hidden layer we are going to use standard Leaky integrate-and-fire neurons. Finally, we are going to use a non-spiking output layer and read classifications out of this by determening the maximum of the output neurons' averaged membrane voltages.



In [6]:
# Create sequential model
serialiser = Numpy("latency_mnist_checkpoints")
network = SequentialNetwork(default_params)
with network:
    # Populations
    input = InputLayer(SpikeInput(max_spikes=BATCH_SIZE * NUM_INPUT),
                                  NUM_INPUT)
    hidden = Layer(Dense(Normal(mean=0.078, sd=0.045)),
                   LeakyIntegrateFire(v_thresh=1.0, tau_mem=20.0,
                                      tau_refrac=None),
                   NUM_HIDDEN, Exponential(5.0))
    output = Layer(Dense(Normal(mean=0.2, sd=0.37)),
                   LeakyIntegrate(tau_mem=20.0, readout="avg_var"),
                   NUM_OUTPUT, Exponential(5.0))

## Compilation
In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, we use the ``EventPropCompiler`` to train with EventProp and specify batch size and how many timesteps to evaluate each example for as well as choosing our optimiser and loss function. Because this is a classification task, we want to use cross-entropy loss and, because our labels are specified in this way (rather than e.g. one-hot encoded), we use the sparse catgorical variant.

In [17]:
compiler = EventPropCompiler(example_timesteps=20,
                         losses="sparse_categorical_crossentropy",
                         optimiser=Adam(1e-2), batch_size=BATCH_SIZE)
compiled_net = compiler.compile(network)

## Training
Now we will train the model for 10 epochs using our compiled network. To verify its performance we take 10% of the training data as a validation split and add an additional callback to checkpoint weights every epoch.



In [18]:
with compiled_net:
    # Evaluate model on numpy dataset
    callbacks = ["batch_progress_bar", Checkpoint(serialiser)]
    compiled_net.train({input: train_spikes},
                       {output: mnist.train_labels()},
                       num_epochs=15, shuffle=True,
                       validation_split=0.1,
                       callbacks=callbacks)

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

## Evaluate
Load weights checkpointed from last epoch:

In [ ]:
network.load((14,), serialiser)

Create an ``InferenceCompiler`` and compile network for inference:

In [ ]:
compiler = InferenceCompiler(evaluate_timesteps=20,
                             reset_in_syn_between_batches=True,
                             batch_size=BATCH_SIZE)
compiled_net = compiler.compile(network)

Encode *test set* using the same log-latency encoding and evaluate it:

In [15]:
test_spikes = linear_latency_encode_data(mnist.test_images(), 20.0)
with compiled_net:
    compiled_net.evaluate({input: test_spikes},
                          {output: mnist.test_labels()})

  0%|          | 0/79 [00:00<?, ?it/s]